# Empirical Analysis of Market Capitalization Weighted Index (MWI) vs. Equal Weighted Index (EWI) Performance for S&P 500

This notebook presents an empirical analysis of the performance of Market Capitalization Weighted Indexing (MWI) and Equal Weighted Indexing (EWI) strategies applied to a subset of the S&P 500. The objective is to explore why the EWI strategy often outperforms the MWI strategy.

For the sake of stability and simplicity in this analysis, we are working with the 411 companies that have continuously been part of the S&P 500 from January 2006 to August 2023. This eliminates any effects due to companies entering or leaving the index during this period.

We begin by fetching the necessary data for these companies using open-source Python package like `yfinance` and web scraping techniques. All the data used in this analysis is freely available and no proprietary or paid sources were used.

Next, we build MWI and EWI portfolios from this data, calculating returns and other relevant metrics.Please note that we do not consider transaction fees in these custum portfolios to avoid additional complexity. Furthermore, for the rebalancing of both portfolios, we perform it after the close of the third Friday in March, June, September, and December. 

Through exploratory data analysis, we delve into the distribution characteristics of individual stock returns, focusing on their skewness as a potential explanatory factor for the observed performance difference. We also visualize various aspects of our portfolios to better understand their composition and performance over time.

Let's begin

## Step 1: Fetching the Tickers of S&P 500 Constituents

The function `get_sp500_tickers()` fetches the ticker symbols of all companies that are currently constituents of the S&P 500 index.

This is done by web scraping the Wikipedia page that lists the S&P 500 constituents. We are using the pandas `read_html` function which conveniently fetches tabular data from an HTML page and returns it in the form of a DataFrame. After fetching the data, we just extract the ticker symbols and return them as a list. 



In [3]:
import pandas as pd 

def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    table = pd.read_html(url, header=0)
    df = table[0]
    return list(df.Symbol)

After running this function, we will have a list of ticker symbols for all current S&P 500 constituents stored in the `tickers` variable.


In [4]:
tickers = get_sp500_tickers()

In [8]:
sorted(tickers)

['A',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ATO',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXON',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BDX',
 'BEN',
 'BF.B',
 'BG',
 'BIIB',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BMY',
 'BR',
 'BRK.B',
 'BRO',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDAY',
 'CDNS',
 'CDW',
 'CE',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COO',
 'COP',
 'COST',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTL

# Fetching Historical Market Capitalization Data 

To construct our **Market Capitalization Weighted Index (MWI)**, we need to collect historical daily market capitalization data for each constituent of the S&P 500. Sourcing this type of data can often be challenging due to access restrictions and cost constraints. After thorough exploration, I found a freely available API endpoint on StockAnalysis.com that supplies this precise information.

This endpoint was discovered by examining the Network tab in the browser's Developer Tools while interacting with the `stockanalysis.com` website. This allowed us to observe the API calls made by the web application.

The API endpoint we identified is `https://stockanalysis.com/api/symbol/s/{symbol}/marketcap?t=price`, where `{symbol}` the ticker symbol of a stock we're interested in. This API call returns a JSON object with a `status` field, indicating the success or failure of the request, and a `data` field containing the actual market cap data. The `data` field is an array of arrays, where each inner array consists of a Unix timestamp (representing the date) and the market cap for the specified stock on that date.

To effectively gather data for all the stocks in our list, I implemented a concurrent fetching system `concurrent.futures` module. This system allows us to send multiple API requests simultaneously, significantly reducing the total time required to gather all our necessary data.



In the following code, we will pull historical market capitalization data for all currently listed stocks in the S&P 500 index :




In [9]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tenacity import retry, stop_after_attempt, wait_fixed

@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def get_market_cap(symbol):
    url = f"https://stockanalysis.com/api/symbol/s/{symbol}/marketcap?t=price"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame({
        'Date': [pd.to_datetime(x[0], unit='ms') for x in data['data']],
        f'Market Cap {symbol}': [x[1] for x in data['data']]
    })

def get_market_caps(tickers):
    first_ticker = tickers.pop(0)
    master_df = get_market_cap(first_ticker)

    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_ticker = {executor.submit(get_market_cap, ticker): ticker for ticker in tickers}
        for future in as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                df = future.result()
                master_df = master_df.merge(df, on='Date', how='outer')
            except Exception as e:
                print(f"Error occurred while getting market cap data for {ticker}: {e}")

    return master_df